# 학습활동. [난이도 상] 지도학습 기반, G&I 문장 경계선 구분하기 위한 결정요소 파악하기

nltk 교재 6장의 2.1 Sentence Segmentation에 있는 예제를 학습하고, 다음 차례에 따라, G&I에 적용해 보자.
1. sentence_tokenized의 폴더(Genomics-Informatics-Corpus-master\GNI Corpus 1.0\sentence_tokenized)에 있는 파일을, 임의로 8:2 정도로 나누어, G&I train set와 G&I test set을 준비해 보자. 
2. G&I 코퍼스에 대하여, white space를 지우고, raw 버젼과 인덱싱 작업 등 약간의 사전 작업을 해야 할 것임 (문제 설명 동영상 참고할 것)
3. G&I test set에 대하여, 1) 교재의 brown 코퍼스를 기반하여, 트레이닝 시킨 모델(Naive Bayesian)과, 2) G&I train set에 기반하여 트레이닝 시킨 모델(Naive Bayesian)을 각각 적용하여, evaluation 결과를 비교해 보자.
4. 두개의 모델 각각에 대해서, show_most_informative() 함수를 사용해서 결과를 살펴 보고, 그 의미를 말로 설명해 볼 것.
- sentence_tokenized 원본 폴더의 데이터에도 에러가 있을 수 있으므로, 적어도 2개 이상의 GNI 논문에 대해서 문장분리 작업을 눈으로 확인하여 Test Set을 점검하고 evaluation 결과를 confirm 해 줄 것.

In [4]:
import nltk
from nltk.corpus import brown

In [5]:
def segment_sentences(words,classifier):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [6]:
# G&I 기반
corpus_root = "/Users/jianny/Genomics-Informatics-Corpus/GNI Corpus 1.0/sentence_tokenized/"
gniCorpus = nltk.corpus.PlaintextCorpusReader(corpus_root, ".*\.txt", encoding="utf-8")

gniRaws = gniCorpus.raw().split('\n')
gniRaws = [nltk.word_tokenize(x) for x in gniRaws]
gniSents = [x for x in gniRaws if len(x) > 5] # 임의로 5개 정도 이하인 줄 단위 문장들은 버림

#brown corpus 기반
brownCorpus = nltk.corpus.brown.sents()
brownSents = brownCorpus

In [7]:
len(gniSents) # raw 버젼과 sentence 단위 버젼

113090

In [8]:
len(brownSents)

57340

In [9]:
gniSents[:10]

[['Editor',
  '’',
  's',
  'Introduction',
  'to',
  'This',
  'Issue',
  '(',
  'G',
  '&',
  'I',
  '15:3',
  ',',
  '2017',
  ')',
  '.'],
 ['Genomics',
  '&',
  'Informatics',
  'Vol',
  '.',
  '15',
  ',',
  'No',
  '.',
  '3',
  ',',
  '2017eISSN2234-0742Genomics',
  'Inform',
  '2017',
  ';',
  '15',
  '(',
  '3',
  ')',
  ':81https',
  ':',
  '//doi.org/10.5808/GI.2017.15.3.81*Corresponding',
  'author',
  ':',
  'Tel',
  ':',
  '+82-2-2258-7343',
  ',',
  'Fax',
  ':',
  '+82-2-537-0572',
  ',',
  'E-mail',
  ':',
  'yejun',
  '@',
  'catholic.ac.kr'],
 ['Yeun-Jun',
  'Chung*Precision',
  'Medicine',
  'Research',
  'Center',
  ',',
  'College',
  'of',
  'Medicine',
  ',',
  'The',
  'Catholic',
  'University',
  'of',
  'Korea',
  ',',
  'Seoul',
  '06591',
  ',',
  'KoreaIn',
  'this',
  'issue',
  ',',
  'Drs',
  '.',
  'Sun',
  'Ho',
  'Lee',
  'and',
  'Wung',
  'Joo',
  'Song',
  'report',
  'copy',
  'number',
  'variations',
  'in',
  '42',
  'Korean',
  'patients',


In [10]:
brownSents[:10]

[['The',
  'Fulton',
  'County',
  'Grand',
  'Jury',
  'said',
  'Friday',
  'an',
  'investigation',
  'of',
  "Atlanta's",
  'recent',
  'primary',
  'election',
  'produced',
  '``',
  'no',
  'evidence',
  "''",
  'that',
  'any',
  'irregularities',
  'took',
  'place',
  '.'],
 ['The',
  'jury',
  'further',
  'said',
  'in',
  'term-end',
  'presentments',
  'that',
  'the',
  'City',
  'Executive',
  'Committee',
  ',',
  'which',
  'had',
  'over-all',
  'charge',
  'of',
  'the',
  'election',
  ',',
  '``',
  'deserves',
  'the',
  'praise',
  'and',
  'thanks',
  'of',
  'the',
  'City',
  'of',
  'Atlanta',
  "''",
  'for',
  'the',
  'manner',
  'in',
  'which',
  'the',
  'election',
  'was',
  'conducted',
  '.'],
 ['The',
  'September-October',
  'term',
  'jury',
  'had',
  'been',
  'charged',
  'by',
  'Fulton',
  'Superior',
  'Court',
  'Judge',
  'Durwood',
  'Pye',
  'to',
  'investigate',
  'reports',
  'of',
  'possible',
  '``',
  'irregularities',
  "''",
 

In [11]:
gniTokens = []
gniBoundaries = set()
gniOffset = 0

#gni 경계선(바운더리)를 만드는 과정
for sent in gniSents:
        gniTokens.extend(sent)
        gniOffset += len(sent)
        gniBoundaries.add(gniOffset-1)

In [12]:
brownTokens = []
brownBoundaries = set()
brownOffset = 0

#brown 경계선(바운더리)를 만드는 과정
for sent in brownSents:
        brownTokens.extend(sent)
        brownOffset += len(sent)
        brownBoundaries.add(brownOffset-1)

In [13]:
#boundary를 이용하여 featuresets을 만드는 함수
def punct_features(tokens, i):
        return {'next-word-capitalized': tokens[i+1][0].isupper(),
                'prev-word': tokens[i-1].lower(),
                'punct': tokens[i],
                'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [14]:
list(gniBoundaries)

[262144,
 2359296,
 2621442,
 524291,
 786437,
 1310731,
 2883596,
 15,
 1048592,
 1835028,
 524311,
 786460,
 1310749,
 1572892,
 1048607,
 2097180,
 2883612,
 262178,
 2359330,
 786473,
 1835051,
 2097198,
 1310769,
 2883633,
 52,
 524340,
 786486,
 3145781,
 262202,
 2097212,
 1572929,
 2359361,
 1835077,
 1048648,
 2097224,
 786511,
 1572944,
 2097233,
 3145814,
 1310808,
 2621528,
 262236,
 524381,
 2883678,
 2097248,
 1048674,
 3145827,
 1835108,
 1572972,
 2621548,
 1310830,
 524405,
 2359415,
 786552,
 3145849,
 2883711,
 2097282,
 1310851,
 1835139,
 262278,
 786567,
 1048710,
 524430,
 2621582,
 1048723,
 1573012,
 3145875,
 2883734,
 2359449,
 2097308,
 1835169,
 1310885,
 2359463,
 3145895,
 786603,
 524461,
 262319,
 1573040,
 2097329,
 2883760,
 1048756,
 1835191,
 2621624,
 786619,
 1573061,
 1310918,
 2883781,
 262344,
 524489,
 2359496,
 2097358,
 1835216,
 1048785,
 1573074,
 2097365,
 786646,
 2883801,
 1835229,
 262377,
 1310954,
 2097385,
 2359530,
 524525,
 314596

In [15]:
list(brownBoundaries)

[786444,
 917517,
 131086,
 262160,
 524305,
 655380,
 24,
 393240,
 131100,
 262174,
 524319,
 393251,
 786468,
 1048615,
 262185,
 917546,
 655403,
 393263,
 655407,
 262196,
 917558,
 1048632,
 786489,
 393277,
 131134,
 262207,
 524349,
 917569,
 67,
 786503,
 1048647,
 262219,
 655438,
 1048654,
 131155,
 786516,
 262229,
 786520,
 1048664,
 1048668,
 262239,
 786528,
 102,
 393319,
 262249,
 131178,
 524398,
 1048686,
 262259,
 786549,
 393336,
 524410,
 262269,
 131206,
 917638,
 655497,
 139,
 1048715,
 524431,
 1048720,
 1048725,
 262296,
 131226,
 786587,
 393372,
 262306,
 163,
 524450,
 655523,
 786596,
 1048743,
 917675,
 1048747,
 786605,
 187,
 262332,
 131261,
 786620,
 917697,
 1048769,
 655558,
 1048776,
 393420,
 917708,
 786639,
 262353,
 524502,
 1048792,
 262362,
 655579,
 131295,
 917728,
 131297,
 786659,
 393444,
 230,
 262374,
 232,
 524519,
 917738,
 1048806,
 1048816,
 393457,
 786673,
 917748,
 1048827,
 655614,
 131327,
 917758,
 258,
 524550,
 1048838,
 2

In [16]:
gniFeaturesets = [(punct_features(gniTokens, i), (i in gniBoundaries))
                  for i in range(1, len(gniTokens)-1)
                  if gniTokens[i] in '.?!']

brownFeaturesets = [(punct_features(brownTokens, i), (i in brownBoundaries))
                    for i in range(1, len(brownTokens)-1)
                    if brownTokens[i] in '.?!']

In [17]:
gniFeaturesets

[({'next-word-capitalized': True,
   'prev-word': ')',
   'punct': '.',
   'prev-word-is-one-char': True},
  True),
 ({'next-word-capitalized': False,
   'prev-word': 'vol',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': False,
   'prev-word': 'no',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': 'drs',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': ')',
   'punct': '.',
   'prev-word-is-one-char': True},
  False),
 ({'next-word-capitalized': True,
   'prev-word': 'analysis',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': 'patients',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': 'group',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,


In [18]:
brownFeaturesets

[({'next-word-capitalized': True,
   'prev-word': 'place',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'conducted',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': False,
   'prev-word': 'jr.',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': "''",
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': "''",
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': "''",
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': "''",
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': "''",
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-w

In [19]:
#gni
gniSize = int(len(gniFeaturesets) * 0.1) #9:1정도로 나누어 train과 test data 분리
gni_train_set1, test_set1 = gniFeaturesets[gniSize:], gniFeaturesets[:gniSize]

#train
gniClassifier = nltk.NaiveBayesClassifier.train(gni_train_set1) #gni train set에 기반

#test
gniTest = nltk.classify.accuracy(gniClassifier, test_set1)

#evaluation
print("Sentence Segmentation based G&I:", gniTest)

Sentence Segmentation based G&I: 0.9555774925962488


In [20]:
#brown corpus
brownSize = int(len(brownFeaturesets) * 0.1) #9:1정도로 나누어 train과 test data 분리
brown_train_set2, test_set2 = brownFeaturesets[brownSize:], gniFeaturesets[:gniSize]

#train
brownClassifier = nltk.NaiveBayesClassifier.train(brown_train_set2) #brown corpus 기반

#test
brownTest = nltk.classify.accuracy(brownClassifier, test_set2)

#evaluation
print("Sentence Segmentation based Brown corpus:", brownTest)

Sentence Segmentation based Brown corpus: 0.9698465404289689


In [21]:
#evaluation
print("Sentence Segmentation based G&I:", gniTest)
print("Sentence Segmentation based Brown corpus:", brownTest)

Sentence Segmentation based G&I: 0.9555774925962488
Sentence Segmentation based Brown corpus: 0.9698465404289689


In [22]:
gniClassifier.show_most_informative_features()

Most Informative Features
                   punct = '?'             False : True   =   3409.5 : 1.0
               prev-word = 'etc'           False : True   =    924.8 : 1.0
               prev-word = 'rhythms'       False : True   =     64.0 : 1.0
               prev-word = 'angiosperms'   False : True   =     64.0 : 1.0
               prev-word = 'condition'     False : True   =     64.0 : 1.0
               prev-word = 'medicine'      False : True   =     49.8 : 1.0
               prev-word = 'authorization'  False : True   =     45.7 : 1.0
               prev-word = 'al'             True : False  =     44.0 : 1.0
               prev-word = 'partition'     False : True   =     38.4 : 1.0
               prev-word = 'i'             False : True   =     38.4 : 1.0


In [23]:
brownClassifier.show_most_informative_features()

Most Informative Features
               prev-word = 'what'          False : True   =     27.0 : 1.0
               prev-word = ')'              True : False  =     17.3 : 1.0
                   punct = '?'             False : True   =     15.6 : 1.0
                   punct = '!'             False : True   =     15.6 : 1.0
               prev-word = 'huh'           False : True   =     13.7 : 1.0
   prev-word-is-one-char = True             True : False  =     12.1 : 1.0
               prev-word = 'i'             False : True   =     10.9 : 1.0
   next-word-capitalized = False           False : True   =     10.4 : 1.0
               prev-word = 'slater'        False : True   =      9.8 : 1.0
               prev-word = 'morale'        False : True   =      9.8 : 1.0
